## Cropping from Raw to extract eyes open and eyes closed recording
#### Input: *.edf with recording data and the .csv containing timestamps for resting recording
####  Output: *-resting_EC-raw.fif, *resting_EO-raw.fif, *-noise_EO-raw.fif

In [2]:
import sys
#sys.path.append('/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/')
sys.path.append('/media/sb10flpc002/08e63286-43ce-4f61-9491-1ed048c96f20/Rachel Wu/eeg-projects/Code/eeg_toolkit/')
import os
import mne
import pandas as pd
#%conda install openpyxl -y
import eeg_toolkit
from eeg_toolkit import utils, preprocess, source_localization

0 files missing from root.txt in /home/sb10flpc002/mne_data/MNE-fsaverage-data
0 files missing from bem.txt in /home/sb10flpc002/mne_data/MNE-fsaverage-data/fsaverage


### Testing the get_cropped_resting_EEGs function

In [32]:
save_path_resting = '/media/sb10flpc002/08e63286-43ce-4f61-9491-1ed048c96f20/Rachel Wu/eeg-projects/Chronic Low Back Pain Study/Data'
csv_path = '/media/sb10flpc002/08e63286-43ce-4f61-9491-1ed048c96f20/Rachel Wu/eeg-projects/Chronic Low Back Pain Study/Data/Eyes Timestamps'
file_path = '/run/user/1000/gvfs/smb-share:server=research-cifs.nyumc.org,share=research/jwanglab/jwanglabspace/George Kenefati/Chronic Low Back Pain Study/Data/Processed Data/008_preprocessed-raw.fif'
raw_file = mne.io.read_raw_fif(file_path, preload = False)
preprocess.get_cropped_resting_EEGs("008", raw_file, csv_path, save_path_resting)

Eyes closed is not longer than 3 mins. Length of EC reading is: 16.94749999999999


(<Raw | 008_preprocessed-raw.fif, 32 x 6780 (16.9 s), ~43 kB, data not loaded>,
 <Raw | 008_preprocessed-raw.fif, 32 x 36001 (90.0 s), ~43 kB, data not loaded>,
 <Raw | 008_preprocessed-raw.fif, 32 x 60001 (150.0 s), ~43 kB, data not loaded>)

In [13]:
eyes_closed = mne.io.read_raw_fif("/media/sb10flpc002/08e63286-43ce-4f61-9491-1ed048c96f20/Rachel Wu/eeg-projects/Chronic Low Back Pain Study/Data/037_eyes_closed-raw.fif", preload = False)
eyes_closed.plot()

2024-01-12 11:14:11,250 - qdarkstyle - INFO - QSS file successfully loaded.
2024-01-12 11:14:11,251 - qdarkstyle - INFO - Found version patches to be applied.
2024-01-12 11:14:11,252 - qdarkstyle - INFO - Found application patches to be applied.


In [14]:
eyes_open = mne.io.read_raw_fif("/media/sb10flpc002/08e63286-43ce-4f61-9491-1ed048c96f20/Rachel Wu/eeg-projects/Chronic Low Back Pain Study/Data/037_eyes_open-raw.fif", preload = False)
eyes_open.plot()

2024-01-12 11:15:19,982 - qdarkstyle - INFO - QSS file successfully loaded.
2024-01-12 11:15:19,983 - qdarkstyle - INFO - Found version patches to be applied.
2024-01-12 11:15:19,983 - qdarkstyle - INFO - Found application patches to be applied.


In [27]:
save_path_resting = '/media/sb10flpc002/08e63286-43ce-4f61-9491-1ed048c96f20/Rachel Wu/eeg-projects/Chronic Low Back Pain Study/Data'
csv_path = '/media/sb10flpc002/08e63286-43ce-4f61-9491-1ed048c96f20/Rachel Wu/eeg-projects/Chronic Low Back Pain Study/Data/Eyes Timestamps'
file_path = '/run/user/1000/gvfs/smb-share:server=research-cifs.nyumc.org,share=research/jwanglab/jwanglabspace/George Kenefati/Chronic Low Back Pain Study/Data/Processed Data/044_preprocessed-raw.fif'
raw_file = mne.io.read_raw_fif(file_path, preload = False)
preprocess.get_cropped_resting_EEGs("044", raw_file, csv_path, save_path_resting)

(<Raw | 044_preprocessed-raw.fif, 64 x 72001 (180.0 s), ~80 kB, data not loaded>,
 <Raw | 044_preprocessed-raw.fif, 64 x 36001 (90.0 s), ~80 kB, data not loaded>,
 <Raw | 044_preprocessed-raw.fif, 64 x 60001 (150.0 s), ~80 kB, data not loaded>)

In [28]:
save_path_resting = '/media/sb10flpc002/08e63286-43ce-4f61-9491-1ed048c96f20/Rachel Wu/eeg-projects/Chronic Low Back Pain Study/Data'
csv_path = '/media/sb10flpc002/08e63286-43ce-4f61-9491-1ed048c96f20/Rachel Wu/eeg-projects/Chronic Low Back Pain Study/Data/Eyes Timestamps'
file_path = '/run/user/1000/gvfs/smb-share:server=research-cifs.nyumc.org,share=research/jwanglab/jwanglabspace/George Kenefati/Chronic Low Back Pain Study/Data/Processed Data/C6._preprocessed-raw.fif'
raw_file = mne.io.read_raw_fif(file_path, preload = False)
preprocess.get_cropped_resting_EEGs("C6.", raw_file, csv_path, save_path_resting)

(<Raw | C6._preprocessed-raw.fif, 64 x 72001 (180.0 s), ~80 kB, data not loaded>,
 <Raw | C6._preprocessed-raw.fif, 64 x 48001 (120.0 s), ~80 kB, data not loaded>,
 <Raw | C6._preprocessed-raw.fif, 64 x 72001 (180.0 s), ~80 kB, data not loaded>)

In [9]:
# Settings
times_tup,time_win_path = preprocess.get_time_window(5)

# Read in csv files with timestamps for eyes open/closed/stop
csv_path = '../../../Data/Eyes Timestamps/'

# Read in processed raw data
data_path = '../../Data/Processed Data/'
epo_path = '../../Data/Processed Data/' + time_win_path
save_path_resting = '../../Data/Processed Data/'
save_path_cont = '../../Data/STC_Continuous/'
save_path_zepo = '../../Data/STC_Z_scored_Epochs/' + time_win_path

if not os.path.exists(save_path_resting):
    os.makedirs(save_path_resting)
    
if not os.path.exists(save_path_cont):
    os.makedirs(save_path_cont)

if not os.path.exists(save_path_zepo):
    os.makedirs(save_path_zepo)

# Noise cov: crop raw during eyes-open resting condition
noise_cov_win = 5.5, 7.5 # rest_min, rest_max

[-2.5,0.0,2.5]


In [6]:
# viable subjects
sub_ids = utils.import_subs(data_path)
# select only 64ch subs and turn into set
sub_ids = sub_ids[7:]
print(sub_ids)

['018', '020', '021', '022', '023', '024', '027', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', 'C1.', 'C2.', 'C3.', 'C5.', 'C6.', 'C7.', 'C8.', 'C9.', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C24', 'C25', 'C26', 'C27']


In [7]:
print(f"Chronics: {len([el for el in sub_ids if el.startswith('0')])}")
print(f"Controls: {len([el for el in sub_ids if el.startswith('C')])}")
print(f"Total: {len(sub_ids)}")

Chronics: 36
Controls: 25
Total: 61


In [1]:
# Function purpose: Obtain the CSV file with timestamps for resting EEG timeframe
# Inputs: sub_id = subject id of interest, csv_path = file path to the csv
# Outputs: Corresponding csv file for subject of interest
def load_csv(sub_id, csv_path):
    csv_folder = os.listdir(csv_path)
    for file in csv_folder:
        if file.endswith(".csv") and sub_id in file:
            return pd.read_csv(os.path.join(csv_path, file))
    print(f"CSV file with {sub_id} not found in the folder.")
    return None

# Function purpose: Create cropped files and save them.
# Inputs: raw = *raw.fif file, start = beginning timepoint in seconds, stop = ending timepoint in seconds
            # save_path = file path to file for saved cropped data
            # category = name for file (eyes_closed, noise, eyes_open)
# Outputs: cropped file
def save_cropped_data(raw, start, stop, sub_id, save_path, category):
    filename = f"{sub_id}_{category}-raw.fif"
    filepath = os.path.join(save_path, filename)
    cropped = raw.copy().crop(tmin=start, tmax=stop)
    cropped.save(filepath, overwrite=True)
    return cropped

# Function purpose: Create recording of the full resting EEG
# Inputs: sub_id = subject ID ie the patient number, csv_path = file path for the csv with the resting timestamps
        # raw = *{sub_id}...raw.fif file
# Outputs: *raw.fif file with recording for eyes closed only (e.g. 007_eyes_closed-raw.fif)
        # *raw.fif file with recording for noise calibration only (e.g. 007_noise-raw.fif)
        # *raw.fif file with recording for eyes open only (e.g. 007_eyes_open-raw.fif)
def crop_resting(sub_id, csv_path, raw):
    timestamp_csv = load_csv(sub_id, csv_path)
    if timestamp_csv is None: return None

    EC_start, EC_stop, EO_start, EO_stop = timestamp_csv['Seconds'][0:4]
    
    # Establish timestamps assuming enough recorded for 5 mins of eyes open noise = 2 mins, EO = 3 mins
    noise_start = EO_start
    noise_stop = EO_start + 120
    cropped_EO_start = noise_stop
    cropped_EO_stop = noise_stop + 180

    EO_duration = EO_stop - EO_start

    # Adjust durations based on the length of the recording
    if EO_duration < 300: #Resting recording is less than 5 mins
        if EO_duration < 270: #Resting recording is less than 4.5 mins
            noise_stop = EO_start + 60 # Noise is 1 minute 
            cropped_EO_start = noise_stop
            cropped_EO_stop = noise_stop + 180 # EO is 3 minutes
        else:
            noise_stop = EO_start + 90 # Noise is 1.5 mins
            cropped_EO_stop = noise_stop + 180 # EO is 3 minutes

    # Send message if eyes closed is shorter than 3 mins, otherwise default is 3 min eyes closed recording
    if (EC_stop - EC_start) < 180:
        print(f"Eyes closed is not longer than 3 mins. Length of EC reading is: {EC_stop- EC_start}")
    else:
        EC_stop = EC_start + 180

    # Crop and save the cropped raw data to a raw.fif file
    EC_cropped = save_cropped_data(raw, EC_start, EC_stop, sub_id, save_path_resting, 'eyes_closed')
    noise_cropped = save_cropped_data(raw, noise_start, noise_stop, sub_id, save_path_resting, 'noise')
    EO_cropped = save_cropped_data(raw, cropped_EO_start, cropped_EO_stop, sub_id, save_path_resting, 'eyes_open')

    return EC_cropped, noise_cropped, EO_cropped

In [11]:
for sub_id in sub_ids:
    _ = source_localization.to_source(sub_id,data_path,epo_path,
                                  save_path_cont,
                                  save_path_zepo,
                                  roi_names,
                                  times_tup,
                                  noise_cov_win,
                                  include_zepochs=True,
                                  average_dipoles=True)       

%%%%%%%%%%%%%%	11	!! ZEPOCHS !! medialorbitofrontal-rh		%%%%%%%%%%%%%%

C27_preprocessed-epo.fif
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 60 (4 small eigenvalues omitted)
    Computing noise-normalization factors (dSPM)...
[done]
Picked 64 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 52
combining the current components...
Processing epoch : 2 / 52
combining the current components...
Processing epoch : 3 / 52
combining the current components...
Processing epoch : 4 / 52
combining the current components...
Processing epoch : 5 / 52
combining the current components...
Processing epoch : 6 / 52
combining the current components...
Processing epoch : 7 / 52
combining the current components...
Processing epoch : 8